# Case study 1: Diffusion of fluid pressure and seismicity below Mt. Hood

We will apply our new transient model to study the relation between fluid pressure and seismicity in the crust below an active volcano, Mt. Hood in Oregon, USA. We will follow a publication by Saar and Manga (2003). The central claim of this paper is that there appears to be a correlation between seasonal recharge and watertable changes and seismicity in the upper crust. Saar and Manga suggest that this may be due to the effect of fluid pressure on seismicity. Groundwater recharge increases the watertable, which in turn increases pore fluid pressure in the crust. High fluid pressure means a lower effective normal stress on fault planes, which makes faults more likely to fail and generate seismic activity. We will use our 1D model to model the seasonal change in fluid pressure in the crust, and will try to quantify the magnitude of changes at a depth of 4500 m, where seismic activity is thought to originate.

## Workflow

1. Experiment with the timestep size to get a model that is stable
2. Model the effect of an instantaneous increase in recharge and record how long it takes before it reaches the depth where seismic activity occurs (4500 m below the surface)
3. Adjust hydraulic conductivity and storativity to find the optimal values that correspond to the phase shift between the refharge events and the spike in seimsic activity (151 days)
4. Implement a periodic boundary condition to simulate seasonal recharge and calculate the pore-pressure change at depth due to seismic activity

## 1. Keeping the model stable

The solution that we are using here is a so-called explicit finite difference solution. This is, next to the steady-state solution, the easiest and most intuitive approach to solving partial differential equations and simulating physical processes. However, explicit solutions have one drawback, the solution can become numerically unstable at large timestep sizes ($\Delta t$). We can test this experimentally. Increase the value of the timestep `dt` and run your model again. Keep on increasing the timestep until you get weird results. Record the timestep at which this occurs. Bonus points for the most artistic figure of unstable model results.

The timestep value at which these solutions become numerically unstable is predictable and has been quantified by three mathematicians from Göttingen, Courant, Friedrichs and Lewy in 1928. The stability condition is therefore also called the CFL or Courant number, and follows:

\begin{equation}
    CFL = \dfrac{q \Delta t}{\Delta x}
\end{equation}

The numerical solution becomes unstable for values of CFL that exceed 1.

**Assignment 1: Run the model several times and adjust the timestep size until the model becomes unstable. Choose a timestep size that is still stable, but large enough for the model to finish relatively fast. What timestep did you choose? Make a figure of an unstable model result.**

## 2. Model the effects of an instantaneous recharge event

We will start by setting up our numerical model and simulating the effect of an instantaneous increase in hydraulic head at the surface. The seasonal recharge at Mt. Hood is approximately equal to a layer of water of 1.5 m. The volcanic rocks at the surface have a specific yield of approximately 0.15. This means that a recharge of 1.5 m will increase the watertable by 1.5/0.15 = 10 m.   

We will try to implement an instantaneous change in hydraulic head at the surface by adjusting the top boundary condition *after* running the steady-state solution first. Add a line where you change the value of the hydraulic head at the surface to 10 m. Note that the variable controlling hydraulic head is an array instead of a single number like in exercise 2. This is because we want to make a boundary condition that varies over time later on.

**Assignment 2 Model the effects of an instantaneous increase in hydraulic head. How long does it take for the change to reach depths of 4500 m, where seismic activity occurs?**

## 3. Adjusting storativity and hydraulic conductivity

Next we will try to make our model more realistic. Instead of a single number for storativity ``S`` calculate the storativity using the specific storage equation given in your lecture handout (lecture 7). Note that we do not directly model the change in watertable. All the nodes in our model are assumed to be located below the watertable, and therefore we can ignore specific yield and assume that storativity is equal to the specific storage.

**Assignment 3: Implement the full equation for storativity in the model code and replace the line that currently assigns a value to ``S``. Use a reasonable value for fluid density, fluid compressibility and the compressibility of the rock matrix. Describe which parameter values you use and which value of storativity this results in. (see equation 16 and the parameter values mentioned below this equation. Use 1000 kg/m3 as density).**

The observed peak in seismicity underneath Mt. Hood shows a delay of approximately 151 days when compared to streamflow peaks at this location. Streamflow can be regarded as an indicator of seasonal groundwater recharge by snowmelt. The offset of peak seismicity may be an indicator of the time it takes for the fluid pressure increase at the surface to affect fluid pressures at 4500 m depth. 

**Assignment 4: Calibrate your model by adjusting ``K`` until the observed delay in increase in hydraulic head at 4500 m compared to the surface equals the observed delay in peak seismicity of ~151 days. Make figures of the model results. Which parameter value for K did you use for the final calibrated model?**

Note that changing K and/or S may make the model unstable again in some cases. This can be fixed by decreasing the timestep size.

## 4. Adding periodic boundary conditions

So far we have modeled the effect of an instantaneous groundwater recharge event. A more realistic way to model the effects of seasonal recharge is to include a periodic change in hydraulic head at the top boundary. Saar and Manga (2003) approximate this by using a cosine function (see equation 8). Uncomment the line where a periodic boundary condition is assigned in this notebook (in the transient parameters box below) and complete the line. Add a new box below, copy-paste the line to calculate the boundary condition and inspect the variable by typing ``print(h0)`` in the next line. Or ``print(h0[:100])`` if you want to inspect the first 100 values for instance. Make sure that you are modelling an increase from 0 to 10 m by checking the min and max values of ``h0`` by typing ``h0.min()`` and ``h0.max()``.

**Assignment 5 Run the model with the new boundary condition and make a figure of the result. What is the seasonal change in hydraulic head at a depth of 4500 m? What is the corresponding change in pore pressure (Pa)? Is this difference high or low compared to the difference between hdyraostatic and lithostatic pressure at these depths? (note, see lecture handouts for lecture on compaction, or ask the dutch guy in front of the classroom)**

# The actual model code:

## import python modules

In [ ]:
import matplotlib

%matplotlib inline

import numpy as np
import matplotlib.pyplot as pl

## Function to calculate steady-state hydraulic head

In [ ]:
def run_steady_state_model(x, dx, K, h0, W, n_iter=20000):
    
    C = (W * dx**2) / K

    h_new = np.ones_like(x) * h0
    h_old = np.ones_like(x) * h0

    # iterative solution steady-state h

    for t in range(n_iter):
    # make sure you indent anything below a for loop

        # set bnd conditions
        # left bnd:
        h_new[0] = h0
        # right bnd:
        h_new[-1] = 0.5 * (C[-1] + h_old[-1] + h_old[-2])

        # middle nodes:
        h_new[1:-1] = 0.5 * (C[1:-1] + h_old[2:] + h_old[:-2])

        h_old = h_new.copy()

    return h_new

## Model parameters

In [ ]:
day = 24.0 * 60.0 * 60.0
year = 365 * 24 * 60 * 60.0

L = 10000.0
dx = 100.0

# top boundary condition for the initial steady-state model
h0 = 0

# hydraulic conductivity
K = 1e-6

## Set up arrays

In [ ]:
# calculate depth of each node
z = np.arange(0, L + dx, dx)

# source term, we keep this at 0 for this exercise:
W = np.zeros_like(z)

## Parameters for transient model runs

In [ ]:
# specify storage coefficient
# for unconfined groundwater flow this is the specific yield
# for confined this is the specific storage
porosity = 0.1
density = 1000.0
g = 9.81
S = 1e-4

# formulate storativity using the compressibilites of water and the rock matrix
# uncomment and complete the following lines
# compressibility_water = ....
# compressibility_rocks = ....
# fluid_density = 1000.0
# g = 9.81
# S = ... a function of porosity, compressibility_water, compressibility_rocks, g, density

# timestep size
dt = 5.0 * day

#total duration
duration = 2.0 * year

# calculate total number of timesteps
n_timesteps = int(duration / dt)

## run steady-state model

We will first run the steady state model. The steady state value of h will be used as an initial value for the transient model runs

In [ ]:
h_steady = run_steady_state_model(z, dx, K, h0, W)

## Set up parameters for the transient model

In [ ]:
# set up array that records timesteps:
time = np.arange(n_timesteps) * dt

# define array to store flux and the variable over time:
n_nodes = len(z)
h = np.zeros((n_timesteps, n_nodes))

# set the steady-state value of u as value for first timestep
h[0] = h_steady

# increase the hydraulic head in the upper node:
h0 = np.zeros(n_timesteps)

# uncomment the following line to change the hydraulic head at the top
# boundary for the transient model runs:
h0[:] = 10

# uncomment the next lines to add a periodic boundary condition
#period = 1 * year
#amplitude = 10.0
#h0 = ... add a variation of equation 8 in the Saar and Manga (2003) paper here, in python language. 
# use the variables time, period, amplitude
# use np.pi for the number pi, np.cos(x) for the cosine of a variable x (replace x with your own desired function or number), 
# and use np.sin() for sine. 

## run the transient model:

In [ ]:
for j in range(1, n_timesteps):
    
    # calculate the flux between nodes
    q = -K * (h[j-1, 1:] - h[j-1, :-1]) / dx

    # set specified variable value at the left-hand node:
    h[j, 0] = h0[j]

    # implement no-flow boundary condition at right-hand side:
    q_right = 0.0
    h[j, -1] = h[j-1, -1] + (dt/S) * (-(q_right - q[-1])/dx) + (dt / S) * W[-1]

    # update nodes in the middle:
    h[j, 1:-1] = h[j-1, 1:-1] + (dt/S)*(-(q[1:] - q[:-1])/dx) + (dt/S) * W[1:-1]

    # print results to screen each 100 timesteps
    if j / 1000 == j / 1000.0:
        print('time = ', ((j * dt) / year), ', min, max value of h = ', h[j].min(), h[j].max())


## Some figures:

### A figure of h vs depth:

In [ ]:
fig, panel = pl.subplots(1, 1)

#  show change in hydraulic head over time
for j in range(0, n_timesteps, int(n_timesteps/20)):
    if j == 0:
        label = 't=0'
    else:
        label = None
    panel.plot(h[j], z, label=label)

label = 't=%0.1f yr' % (duration/year)
panel.plot(h[-1], z, color='black', lw=1.0, label=label)
panel.legend(loc='upper left', fontsize='small')

panel.set_ylabel('Depth (m)')
panel.set_xlabel('Hydraulic head (m)')
panel.set_ylim(L, 0)

fig.savefig('simulated_h_vs_depth.png')

### A figure of h over time for a specific depth

In [ ]:
# add the depths that you would like to show in the figures here (m)
target_depths = np.array([4500])

# add colors for the different depths
target_colors = ['blue', 'orange']

# look up the node for the different target depths
target_nodes = (target_depths / dx).astype(int)

fig, panel = pl.subplots(1, 1)

for target_depth, target_node, color in zip(target_depths, target_nodes, target_colors):
    label = '-%0.0f m' % target_depth
    panel.plot(time/year, h[:, target_node], color=color, label=label)

panel.legend()
panel.set_xlabel('Time (years)')
panel.set_ylabel('Hydraulic head (m)')

fig.savefig('simulated_h_over_time.png')

# References

Courant, R, K Friedrichs, and H Lewy. 1928. “Über Die Partiellen Differenzengleichungen Der Mathematischen Physik.” Mathematische Annalen 100 (1): 32–74. https://doi.org/10.1007/BF01448839  

Saar, M. O. & Manga, M. Seismicity induced by seasonal groundwater recharge at Mt. Hood, Oregon. Earth Planet. Sci. Lett. 214, 605–618 (2003). https://doi.org/10.1016/S0012-821X(03)00418-7

Note, you can find these publications using google scholar: https://scholar.google.com

![Mount Hood](https://raw.githubusercontent.com/ElcoLuijendijk/GEOV212_hydrogeology_exercises/refs/heads/main/fig/mount_hood.jpg)